In [1]:
import serial
import numpy as np
from PIL import Image
import keyboard


In [21]:
#previous data packet
oldData = "1111111111111111"
#indexing
i = 0
#size 256 array
handArray =  np.arange(256)
#3d data set
dataSet = np.empty((0, 16, 16), dtype=int)

In [22]:
# Establish the serial connection
ser = serial.Serial('COM5', 9600)

command = input("type g to begin")
ser.write(command.encode())

while True:
   
    if keyboard.is_pressed('s'):
      stop = input("type g to end")
      np.save('dataSet.npy', dataSet)
      ser.write(stop.encode())
      break
    

    data = ser.readline()
    data = data.decode().strip() # Decode the received bytes and remove extra whitespace
    inputData = [int(data[i:i+1]) for i in range(len(data))]
    if (data != oldData):
      #loops through array with prper indexing
      if i < 8:
        for k in range(8):
          handArray[(8*k)+i] = inputData[k+7]
        i = i + 1;
        if i == 8:
          i = 64

      if i > 8 and i < 72:
        for k in range(8):
          handArray[(8*k)+i] = inputData[k+7]
        i = i + 1;
        if i == 72:
          i = 128
        
      if i > 72 and i < 136:
        for k in range(8):
          handArray[(8*k)+i] = inputData[k+7]
        i = i + 1;
        if i == 136:
          i = 192
      
      if i > 135 and i < 200:
        for k in range(8):
          handArray[(8*k)+i] = inputData[k+7]
        i = i + 1;
        if i == 200:
          i = 0
          handArray = (np.reshape(handArray, (16,16)))
          dataSet = np.append(dataSet,[handArray], axis = 0)
          handArray =  np.arange(256)
      #updates data
      oldData = data
    else:
      #updates data
      oldData = data
      np.save('dataSet.npy', dataSet)
#close the connection
ser.close()


In [23]:
d = np.load('dataSet.npy')


image = Image.new("1", (16, 16))
# Load the pixel data from the array
pixels = [int(pixel) for row in d[10] for pixel in row]

# Put the pixel data into the image
image.putdata(pixels)

# Save the image to a file
image.save("output.png")

In [7]:
ser.close()

In [24]:
for i in range(0, 16):
    print(d[i])
    print()

[[1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
 [1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 